In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/titanic/train.csv')
data

In [ ]:
data.Embarked.unique()
data['Embarked_s']=data['Embarked'].replace('Q',0).replace('C',0).replace('S',1)
data['Embarked_c']=data['Embarked'].replace('S',0).replace('Q',0).replace('C',1)
data['Embarked_q']=data['Embarked'].replace('C',0).replace('S',0).replace('Q',1)

In [ ]:
#data.Sex= data.Sex.map( {'M': 1, 'F': 0, 'O': 3} )
data['Sex'].replace( 'female', 0, inplace=True )
data['Sex'].replace( 'male', 1, inplace=True )
data

In [ ]:
#data=data[data.Embarked!=0]
#data=data[data[('Embarked')].map(data['Embarked'].value_counts())>0]*
#data

In [ ]:
data.isnull().sum()


In [ ]:
#data.Sex=data.Sex.fillna(0)
#data.Age=data.Age.fillna(0)
#data.Cabin=data.Cabin.fillna(0)
#data.Embarked_s=data.Embarked_s.fillna(0)
#data.Embarked_c=data.Embarked_c.fillna(0)
#data.Embarked_q=data.Embarked_q.fillna(0)

In [ ]:
data['Survived']


In [ ]:
data.isnull().sum()



In [ ]:
data.dtypes


In [ ]:
x = data.drop(columns = ['Survived','Ticket','Cabin','PassengerId','Name','Embarked'])

In [ ]:
y = data[['Survived']]


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)

In [ ]:
x.head()

In [ ]:
x.dtypes


In [ ]:
x

In [ ]:
x.Embarked_q.unique()


In [ ]:
nom_cols =[]
ord_cols = []
num_cols=[0,4,5]
null_cols=[1,2,6,7,8]
#Kbin_cols=[0,1,4],
binarizer_cols=[3]

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,KBinsDiscretizer,Binarizer
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.compose import make_column_transformer
from sklearn import set_config
from sklearn.preprocessing import StandardScaler

trans = make_column_transformer((OneHotEncoder(sparse=False),nom_cols),
                                (KNNImputer(n_neighbors=45),null_cols),
                                (OrdinalEncoder(), ord_cols),(StandardScaler(),num_cols)
                                 ,(Binarizer(threshold=55),binarizer_cols),remainder= 'passthrough')
set_config(display= 'diagram')
trans
#trans = make_column_transformer((OneHotEncoder(sparse=False),nom_cols),(SimpleImputer(strategy='mean'),null_cols),
                                 #(OrdinalEncoder(), ord_cols),(StandardScaler(),num_cols),(KBinsDiscretizer(),Kbins_cols)
                                 #,(Binarizer(threshold=55),binarizer_cols),remainder= 'passthrough')
#set_config(display= 'diagram')
#trans

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
algorithm_x=DecisionTreeClassifier(criterion='entropy')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
algorithm = KNeighborsClassifier(15) #
algorithm

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(trans,algorithm)
pipe_x=make_pipeline(trans,algorithm_x)
pipe

In [ ]:
pipe_x

In [ ]:
pipe.fit(x_train,y_train)


In [ ]:
pipe_x.fit(x_train,y_train)


In [ ]:
pred_x=pipe_x.predict(x_test)
pred_x

In [ ]:
from sklearn.metrics import accuracy_score, plot_confusion_matrix
accuracy_score(pred_x,y_test)*100

In [ ]:
pred=pipe.predict(x_test)
pred

In [ ]:
from sklearn.metrics import accuracy_score, plot_confusion_matrix
accuracy_score(pred,y_test)*100

In [ ]:
import matplotlib as plt 
plot_confusion_matrix(pipe,x_test,y_test)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
over= RandomOverSampler()
o_x , o_y = over.fit_resample(x,y)
o_y.value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE
over= RandomOverSampler()
_x , o_y = over.fit_resample(x,y)
o_y.value_counts()

In [ ]:
from imblearn.pipeline import make_pipeline
s = SMOTE()
n_pipe = make_pipeline(trans,s,algorithm)
n_pipe

In [ ]:
accuracy_score(pred,y_test)*100


In [ ]:
from sklearn.linear_model import LogisticRegression as lr
algorithm_2 = lr(solver='liblinear')
pipe_b = make_pipeline(trans,algorithm_2)
pipe_b

In [ ]:
pipe_b.fit(x_train,y_train)


In [ ]:
pred_b = pipe_b.predict(x_test)
pred_b

In [ ]:
accuracy_score(pred_b,y_test)*100

In [ ]:
from sklearn.model_selection import KFold, cross_val_score,StratifiedKFold
kf=StratifiedKFold(n_splits=4)
np.mean(cross_val_score(pipe,x,y,cv=kf,scoring='accuracy')*100) 

In [ ]:
from sklearn.svm import SVC
model1=SVC(kernel='linear')
pipe_1 = make_pipeline(trans,algorithm)
pipe_1
pipe_1.fit(x_train,y_train)
pred_1=pipe.predict(x_test)
accuracy_score(pred_1,y_test)*100

In [ ]:
from sklearn.ensemble import BaggingClassifier
model2=BaggingClassifier(base_estimator=SVC(3))
model2

In [ ]:
y_test.shape

In [ ]:
pipec=make_pipeline(trans,model2)
pipec

In [ ]:
pipec.fit(x_train,y_train)
predc=pipec.predict(x_test)

In [ ]:
predc.shape

In [ ]:
accuracy_score(predc,y_test)*100


In [ ]:
dataA=pd.read_csv('/kaggle/input/titanic/test.csv')
dataA

In [ ]:
dataA['Sex'].replace( 'female', 0, inplace=True )
dataA['Sex'].replace( 'male', 1, inplace=True )

In [ ]:
dataA.isnull().sum()

In [ ]:
#dataA.Age=data1.Age.fillna(0)
dataA.Cabin=dataA.Cabin.fillna(0)
dataA.Fare=dataA.Fare.fillna(0)


In [ ]:
dataA.isnull().sum()

In [ ]:
dataA['Embarked_s']=dataA['Embarked'].replace('Q',0).replace('C',0).replace('S',1)
dataA['Embarked_c']=dataA['Embarked'].replace('S',0).replace('Q',0).replace('C',1)
dataA['Embarked_q']=dataA['Embarked'].replace('C',0).replace('S',0).replace('Q',1)

In [ ]:
dataA.head()

In [ ]:
x1 = dataA.drop(columns = ['Ticket','Cabin','PassengerId','Name','Embarked'])
x1

In [ ]:
x1.isnull().sum()


In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,KBinsDiscretizer,Binarizer
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.compose import make_column_transformer
from sklearn import set_config
from sklearn.preprocessing import StandardScaler

trans = make_column_transformer((OneHotEncoder(sparse=False),nom_cols),
                                (KNNImputer(n_neighbors=45),null_cols),
                                (OrdinalEncoder(), ord_cols),(StandardScaler(),num_cols)
                                 ,(Binarizer(threshold=55),binarizer_cols),remainder= 'passthrough')
set_config(display= 'diagram')
trans

In [ ]:
pred123=pipe_x.predict(x1)
pred123

In [ ]:
submit=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [ ]:
submit['Survived']=pred123
submit.to_csv('submission.csv', index=False)

In [ ]:
submit.Survived.value_counts()